# Hello,

This tutorial is focusing on understanding connection between grpc and sql(we are going to use sqlite for this tutorial).
grpc can be implemented in various languages. we are using python for this tutorial.
Details of grpc can be found at https://grpc.io

Let's begin with installation of grpc packages. 

In [7]:
!make clean

rm -rf .venv 
rm -rf generated
rm -rf protos
rm -rf src


In [8]:
!python2.7 -m virtualenv .venv

New python executable in /home/dheerajbhadani/git/tmp/grpc-sqlite/.venv/bin/python2.7
Also creating executable in /home/dheerajbhadani/git/tmp/grpc-sqlite/.venv/bin/python
Installing setuptools, pip, wheel...done.


In [9]:
!mkdir protos generated src
!touch generated/__init__.py
!find . -not -path '*/\.*'

.
./generated
./generated/__init__.py
./grpc-dbsync.png
./src
./Makefile
./README.md
./protos
./grpc-sqlite.png
./resource
./resource/employee.db
./Main.ipynb
./Main.slides.html
./Client.ipynb
./Server.ipynb
./LICENSE


In [10]:
!.venv/bin/pip install grpcio grpcio-tools

  Using cached https://files.pythonhosted.org/packages/3d/15/b34114198a2bc9c9bb73b21e2b559468a1a68bb28b373d21da6e51d6204f/grpcio-1.15.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/4a/40/17b8e1c649ae9d7a56e5aeba88811ef9cecef1c35ab76899233413aeb955/grpcio_tools-1.15.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c5/db/e56e6b4bbac7c4a06de1c50de6fe1ef3810018ae11732a50f15f62c7d050/enum34-1.1.6-py2-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2d/99/b2c4e9d5a30f6471e410a146232b4118e697fa3ffc06d6a65efde84debd0/futures-3.2.0-py2-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b8/c2/b7f587c0aaf8bf2201405e8162323037fe8d17aa21d3c7dda811b8d01469/protobuf-3.6.1-cp27-cp27mu-manylinux1_x86_64.whl


**After successful installation of the grpc packages, we will start creating the protocol buffer file which contains declaration of service interface and the message which will be used for the client server communication.**

**Create proto file titled employee.proto with the below mentioned content.**

In [ ]:
%%writefile protos/employee.proto

syntax = "proto3";

package employee;

// Interface exported by the server.
service Employee {
    rpc Read(readRequestPB) returns (readResponseListPB) {}
}

message readRequestPB {
    string first_name = 1;
    string last_name = 2;
    string email = 3;
    string gender = 4;
    string ip_address = 5;
    string country = 6;
    string postcode = 7;
}

message readResponsePB {
    string first_name = 1;
    string last_name = 2;
    string email = 3;
    string gender = 4;
    string ip_address = 5;
    string country = 6;
    string postcode = 7;
    int32 id = 8;
}

message readResponseListPB {
    repeated readResponsePB employee = 1;
}

Compile proto file and generate stub for python. Here is the command to generate:

In [ ]:
!./.venv/bin/python -m grpc_tools.protoc -I ./protos/. --python_out=./generated/. --grpc_python_out=./generated/. ./protos/employee.proto

This will generate two files(employee_pb2.py and employee_pb2_grpc.py) under generated folder. 

In [ ]:
!ls -al generated

In [ ]:
!cat generated/employee_pb2.py

In [ ]:
!cat generated/employee_pb2_grpc.py

Now we will start implementing server. create a file under src folder named server.py with the following content:

In [ ]:
%%writefile src/server.py

from __future__ import print_function

import time
import grpc

from generated import employee_pb2_grpc
from concurrent import futures

_ONE_DAY_IN_SECONDS = 60 * 60 * 24

class EmployeeServicer(employee_pb2_grpc.EmployeeServicer):
    def __init__(self):
        pass
    
def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    employee_pb2_grpc.add_EmployeeServicer_to_server(EmployeeServicer(), server)
    server.add_insecure_port('[::]:40084')
    server.start()
    print("Employee server running on 40084...")
    try:
        while True:
            time.sleep(_ONE_DAY_IN_SECONDS)
    except:
        server.stop(0)

if __name__ == '__main__':
    serve()

Client code is as mentioned below in client.py

In [ ]:
%%writefile src/client.py

from __future__ import print_function

import time
import grpc

from generated import employee_pb2, employee_pb2_grpc

class EmployeeClient(object):
    def __init__(self):
        pass

def run():
    channel = grpc.insecure_channel('localhost:40084')
    stub = employee_pb2_grpc.EmployeeStub(channel)
    print(stub)

if __name__ == '__main__':
    run()

So far we have created server and client file with the basic connection. 
###### Lets run section-1 of server and client notebooks.

Client is not calling anything from server yet. So let's implement that...

In [ ]:
class EmployeeServicer(employee_pb2_grpc.EmployeeServicer):
    def Read(self, request, context):
        print("Read employee service called...")
        response_list = employee_pb2.readResponseListPB()
        return response_list

In [ ]:
class EmployeeClient(object):
    def __init__(self):
        pass

    def get_employee(self, stub):
        read_request = employee_pb2.readRequestPB()
        read_request.first_name = "Pauli"
        return stub.Read(read_request)
    
def run():
    channel = grpc.insecure_channel('localhost:40084')
    stub = employee_pb2_grpc.EmployeeStub(channel)

    emp_client = EmployeeClient()
    print(emp_client.get_employee(stub))

After implementing above methods in client and server, server will return an empty list as response.
###### Lets run section-2 of server and client notebooks.
[Server](https://35.242.154.175:8888/notebooks/git/tmp/grpc-sqlite/Server.ipynb)
[Client](https://35.242.154.175:8888/notebooks/git/tmp/grpc-sqlite/Client.ipynb)

Now lets move towards getting the data from database and return that as response to client.
We will create another python file in src folder named employee_orm.py. This file will contain the code as below mentioned: 

In [ ]:
%%writefile src/employee_orm.py

__author__ = 'Dheeraj Bhadani'

import sqlite3

from generated import employee_pb2

class EmployeeORM(object):
    def __init__(self):
        pass

    def run_query(self, query):
        conn = sqlite3.connect('resource/employee.db')
        curs = conn.cursor()
        curs.execute(query)
        return curs

    def select(self, requestPB):
        base_query = 'select * from EMPLOYEE'

        if requestPB.ListFields():
            where_clause = " WHERE "
        condition_count = 0
        for f in requestPB.ListFields():
            if condition_count == 0:
                where_clause += ' {0} = "{1}"'.format(f[0].name, f[1])
            else:
                where_clause += ' AND {0} = "{1}"'.format(f[0].name, f[1])
            condition_count += 1

        query = base_query + where_clause

        query_result = self.run_query(query)

        result = []
        for cur in query_result:
            response = employee_pb2.readResponsePB()
            response.first_name = str(cur[1])
            response.last_name = str(cur[2])
            response.email = str(cur[3])
            response.gender = str(cur[4])
            response.ip_address = str(cur[5])
            response.country = str(cur[6])
            response.postcode = str(cur[7])
            response.id = int(cur[0])
            result.append(response)

        response_list = employee_pb2.readResponseListPB()
        response_list.employee.extend(result)
        return response_list

Connection between orm and server can be implemented as mentioned below:

In [ ]:
import employee_orm

class EmployeeServicer(employee_pb2_grpc.EmployeeServicer):
    def __init__(self):
        self.__emp_orm = employee_orm.EmployeeORM()

    def Read(self, request, context):
        print("Read employee service called...")
        return self.__emp_orm.select(request)

###### Lets run section-3 of server and client notebooks.

To summarize, client sends a request containg protocol buffer in request. Server recieves the request and process the method. Send a request to ORM which get the data from database and return the protocol buffer object. Server respond to client with the protocol buffer object.

![alt text](grpc-sqlite.png "Title")

Whats next>>
implementing other rpc calls like:
    rpc Create(creationRequestPB) returns (creationResponsePB) {}
    rpc Update(updateRequestPB) returns (updateResponsePB) {}
    rpc Delete(deleteRequestPB) returns (deleteResponsePB) {}